In [2]:
import os
os.environ["PYARROW_IGNORE_TIMEZONE"] = "1"
print(os.getcwd())

c:\Users\utcpret\Documents\NF26\nf26_project


In [3]:
import pandas as pd
import numpy as np
from datetime import datetime, date

import pyspark.pandas as ps
from pyspark.sql import SparkSession
from pyspark.sql import Row
from etl_warehouse import *

In [4]:
spark = SparkSession.builder.getOrCreate()

In [5]:
## Définition du fuseau horaire sur paris ##

spark.conf.set("spark.sql.session.timeZone", "Europe/Paris")


TABLE PERSONNEL
pas besoin de la créer de manière journalière car on pouvait l'extraire directement

In [6]:
#BERLIN
psdf_personnel_BERLIN = ps.read_csv('BDD_BGES/BDD_BGES/BDD_BGES_BERLIN/PERSONNEL_BERLIN.txt', index_col='ID_PERSONNEL', sep=';', encoding='latin-1')
sdf_personnel_BERLIN = psdf_personnel_BERLIN.to_spark(index_col='ID_PERSONNEL')

#LONDON
psdf_personnel_LONDON = ps.read_csv('BDD_BGES/BDD_BGES/BDD_BGES_LONDON/PERSONNEL_LONDON.txt', index_col='ID_PERSONNEL', sep=';', encoding='latin-1')
sdf_personnel_LONDON = psdf_personnel_LONDON.to_spark(index_col='ID_PERSONNEL')

#LOSANGELES
psdf_personnel_LOSANGELES = ps.read_csv('BDD_BGES/BDD_BGES/BDD_BGES_LOSANGELES/PERSONNEL_LOSANGELES.txt', index_col='ID_PERSONNEL', sep=';', encoding='latin-1')
sdf_personnel_LOSANGELES = psdf_personnel_LOSANGELES.to_spark(index_col='ID_PERSONNEL')

#NEWYORK
psdf_personnel_NEWYORK = ps.read_csv('BDD_BGES/BDD_BGES/BDD_BGES_NEWYORK/PERSONNEL_NEWYORK.txt', index_col='ID_PERSONNEL', sep=';', encoding='latin-1')
sdf_personnel_NEWYORK = psdf_personnel_NEWYORK.to_spark(index_col='ID_PERSONNEL')

#PARIS
psdf_personnel_PARIS = ps.read_csv('BDD_BGES/BDD_BGES/BDD_BGES_PARIS/PERSONNEL_PARIS.txt', index_col='ID_PERSONNEL', sep=';', encoding='latin-1')
sdf_personnel_PARIS = psdf_personnel_PARIS.to_spark(index_col='ID_PERSONNEL')

#SHANGHAI
psdf_personnel_SHANGHAI = ps.read_csv('BDD_BGES/BDD_BGES/BDD_BGES_SHANGHAI/PERSONNEL_SHANGHAI.txt', index_col='ID_PERSONNEL', sep=';', encoding='latin-1')
sdf_personnel_SHANGHAI = psdf_personnel_SHANGHAI.to_spark(index_col='ID_PERSONNEL')
 
 
 
# Code corrigé pour la dimension personnel
import os
from pyspark.sql.functions import lit, col

# Créer le répertoire DIM_PERSONNEL s'il n'existe pas
os.makedirs("DATA_WAREHOUSE/DIM_PERSONNEL", exist_ok=True)

# Union de tous les DataFrames du personnel
sdf_personnel_all = sdf_personnel_BERLIN.union(sdf_personnel_LONDON) \
    .union(sdf_personnel_LOSANGELES).union(sdf_personnel_NEWYORK) \
    .union(sdf_personnel_PARIS).union(sdf_personnel_SHANGHAI)

# Ajouter la colonne VILLE pour indiquer la provenance
sdf_personnel_berlin_with_city = sdf_personnel_BERLIN.withColumn("VILLE", lit("BERLIN"))
sdf_personnel_london_with_city = sdf_personnel_LONDON.withColumn("VILLE", lit("LONDON"))
sdf_personnel_la_with_city = sdf_personnel_LOSANGELES.withColumn("VILLE", lit("LOSANGELES"))
sdf_personnel_ny_with_city = sdf_personnel_NEWYORK.withColumn("VILLE", lit("NEWYORK"))
sdf_personnel_paris_with_city = sdf_personnel_PARIS.withColumn("VILLE", lit("PARIS"))
sdf_personnel_shanghai_with_city = sdf_personnel_SHANGHAI.withColumn("VILLE", lit("SHANGHAI"))

# Union de tous les DataFrames avec la colonne VILLE
sdf_personnel_all_with_city = sdf_personnel_berlin_with_city.union(sdf_personnel_london_with_city) \
    .union(sdf_personnel_la_with_city).union(sdf_personnel_ny_with_city) \
    .union(sdf_personnel_paris_with_city).union(sdf_personnel_shanghai_with_city)

# Afficher un échantillon
sdf_personnel_all_with_city.show(7)

# Spécifier le chemin complet du fichier CSV (pas seulement le dossier)
dim_personnel_path = "DATA_WAREHOUSE/DIM_PERSONNEL/dimension_personnel.csv"

# Utiliser pandas pour sauvegarder en un seul fichier
personnel_pandas_df = sdf_personnel_all_with_city.toPandas()
personnel_pandas_df.to_csv(dim_personnel_path, index=False)

print(f"Table dimension PERSONNEL créée avec {sdf_personnel_all_with_city.count()} employés")

+--------------------+-------------+----------------+----------+-----------+-----------+-----------+-----------------+-------------+--------+----------+---------+---------+------+-------+------------------+---------------------+-------------------+
|        ID_PERSONNEL|NOM_PERSONNEL|PRENOM_PERSONNEL|  DT_NAISS|VILLE_NAISS| PAYS_NAISS|   NUM_SECU|IND_PAYS_NUM_TELP|NUM_TELEPHONE|NUM_VOIE|  DSC_VOIE|CMPL_VOIE|CD_POSTAL| VILLE|   PAYS|FONCTION_PERSONNEL|TS_CREATION_PERSONNEL|  TS_MAJ_PPERSONNEL|
+--------------------+-------------+----------------+----------+-----------+-----------+-----------+-----------------+-------------+--------+----------+---------+---------+------+-------+------------------+---------------------+-------------------+
|KeyPers_Berlin_12...|        Name0|       FistName0|1993-11-04|      Osaka|      Japan|NS000000000|             NULL|+336##0263188|      57|NomVoie940|     NULL|    #8830|BERLIN|Germany|    Dateningenieur|  2010-09-09 11:22:17|2010-09-09 11:22:17|
|Key

EXECUTION ETL boucle avec tous les jours de 2024.

In [7]:
date1 = "20240710"
date2 = "20240506"

executer_etl_pour_date(date1)
executer_etl_pour_date(date2)

Démarrage de l'ETL pour la date: 20240710
Fichier chargé et filtré: BDD_BGES\BDD_BGES\BDD_BGES_LONDON\BDD_BGES_LONDON_MISSION\MISSION_20240709.txt - 1 lignes
Fichier chargé et filtré: BDD_BGES\BDD_BGES\BDD_BGES_LOSANGELES\BDD_BGES_LOSANGELES_MISSION\MISSION_20240709.txt - 11 lignes
Fichier chargé et filtré: BDD_BGES\BDD_BGES\BDD_BGES_NEWYORK\BDD_BGES_NEWYORK_MISSION\MISSION_20240709.txt - 12 lignes
Fichier chargé et filtré: BDD_BGES\BDD_BGES\BDD_BGES_BERLIN\BDD_BGES_BERLIN_MISSION\MISSION_20240710.txt - 8 lignes
Fichier chargé et filtré: BDD_BGES\BDD_BGES\BDD_BGES_PARIS\BDD_BGES_PARIS_MISSION\MISSION_20240710.txt - 36 lignes
Fichier chargé et filtré: BDD_BGES\BDD_BGES\BDD_BGES_LONDON\BDD_BGES_LONDON_MISSION\MISSION_20240710.txt - 16 lignes
Fichier chargé et filtré: BDD_BGES\BDD_BGES\BDD_BGES_LOSANGELES\BDD_BGES_LOSANGELES_MISSION\MISSION_20240710.txt - 18 lignes
Fichier chargé et filtré: BDD_BGES\BDD_BGES\BDD_BGES_NEWYORK\BDD_BGES_NEWYORK_MISSION\MISSION_20240710.txt - 13 lignes
Fichie